# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 9.3.2
- **Date :** 11/11/2023
- **Heure :** 14:00
- **Données :** `ready_WindGustDir_WindGustSpeed_WindGustDirNum.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** ROCKET

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Fred/ready_WindGustDir_WindGustSpeed_WindGustDirNum.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,RainTomorrow,year,month,day,LocationNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,13.3,18.5,0.0,3.8,8.9,4.0,17.0,82.0,63.0,1017.9,...,0,2008,12,1,36,1.963495,3.926991,-35.024782,117.883608,5
2008-12-02,14.7,21.0,0.0,5.4,9.1,6.0,17.0,64.0,71.0,1012.8,...,0,2008,12,2,36,0.785398,5.890486,-35.024782,117.883608,5
2008-12-03,16.5,24.0,0.2,4.8,10.0,19.0,24.0,36.0,73.0,999.8,...,1,2008,12,3,36,0.392699,3.534292,-35.024782,117.883608,5
2008-12-04,14.1,20.6,7.8,7.0,7.2,13.0,41.0,69.0,81.0,1006.7,...,0,2008,12,4,36,1.570796,3.534292,-35.024782,117.883608,5
2008-12-05,13.0,17.8,0.0,3.6,4.9,13.0,19.0,73.0,63.0,1015.3,...,0,2008,12,5,36,3.926991,3.926991,-35.024782,117.883608,5


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2996 entries, 2008-12-01 to 2017-06-25
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MinTemp        2996 non-null   float64
 1   MaxTemp        2996 non-null   float64
 2   Rainfall       2996 non-null   float64
 3   Evaporation    2996 non-null   float64
 4   Sunshine       2996 non-null   float64
 5   WindSpeed9am   2996 non-null   float64
 6   WindSpeed3pm   2996 non-null   float64
 7   Humidity9am    2996 non-null   float64
 8   Humidity3pm    2996 non-null   float64
 9   Pressure9am    2996 non-null   float64
 10  Pressure3pm    2996 non-null   float64
 11  Cloud9am       2996 non-null   float64
 12  Cloud3pm       2996 non-null   float64
 13  Temp9am        2996 non-null   float64
 14  Temp3pm        2996 non-null   float64
 15  RainToday      2996 non-null   int64  
 16  RainTomorrow   2996 non-null   int64  
 17  year           2996 non-null   int64  
 18

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un transformateur ROCKET

from sktime.transformations.panel.rocket import Rocket

rocket = Rocket()

In [10]:
# Récupération des paramètres initiaux du transformateur ROCKET

rocket.get_params()

{'n_jobs': 1, 'normalise': True, 'num_kernels': 10000, 'random_state': None}

In [11]:
# Ajustement du transformateur ROCKET au jeu d'entraînement

rocket.fit(X_train)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Rocket()

In [12]:
# Récupération des paramètres ajustés du transformateur ROCKET

rocket.get_fitted_params()

{}

In [13]:
# Application du transformateur ROCKET au jeu d'entraînement

X_train_transform = rocket.transform(X_train)

In [14]:
# Instanciation d'un modèle classificateur linéaire

from sklearn.linear_model import RidgeClassifierCV

clf_rocket_ridge = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10)) ## Pour ce paramétrage initial, nous suivons l'exemple fourni par `sktime`, qui fixe 10 alphas, de -3 à 3, à équidistance sur l'échelle logarithmique.

In [15]:
# Récupération des paramètres initiaux du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [16]:
# Entraînement du modèle

clf_rocket_ridge.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [17]:
# Récupération des paramètres ajustés du modèle

clf_rocket_ridge.get_params()

{'alphas': array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
        4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
        2.15443469e+02, 1.00000000e+03]),
 'class_weight': None,
 'cv': None,
 'fit_intercept': True,
 'scoring': None,
 'store_cv_values': False}

In [18]:
# Application du transformateur ROCKET au jeu de test

X_test_transform = rocket.transform(X_test)

In [19]:
# Réalisation des prédictions

y_pred = clf_rocket_ridge.predict(X_test_transform)

## 5. Évaluation

In [20]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0
Classe réelle,
0,344
1,155


In [21]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      1.00      0.82       344
           1       0.00      0.00      0.00       155

    accuracy                           0.69       499
   macro avg       0.34      0.50      0.41       499
weighted avg       0.48      0.69      0.56       499



/Users/omarchoa/anaconda3/envs/datascientest_project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/omarchoa/anaconda3/envs/datascientest_project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/omarchoa/anaconda3/envs/datascientest_project/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh